In [3]:
# @title
import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from IPython.display import display, clear_output

# URLs dos arquivos no GitHub (raw)
url_mapeamento = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/PLANO%20DE%20AÇÃO%20-%20BPA%20-%20FINAL.xlsx"

def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

def converter_arquivo(uploaded_file):
    try:
        # Carrega arquivos modelo e mapeamento
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx", engine='openpyxl')
        modelo = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx", engine='openpyxl')

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]['content']
        df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='openpyxl')

        # Renomeia colunas do BPA
        col_map = dict(zip(mapeamento['Coluna_Origem'], mapeamento['Coluna_Destino']))
        df_convertido = df_bpa.rename(columns=col_map)

        # Cria DataFrame final baseado no modelo
        df_final = modelo.copy()

        # Preenche as colunas do BPA convertidas
        for col in df_final.columns:
            if col in df_convertido.columns:
                df_final[col] = df_convertido[col]

        nome_saida = "BPA_Convertido.xlsx"
        df_final.to_excel(nome_saida, index=False, engine='openpyxl')
        print("✅ Conversão concluída com sucesso!")
        files.download(nome_saida)
    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

def interface():
    upload_button = widgets.FileUpload(accept=".xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

interface()


FileUpload(value={'PLANO DE AÇÃO - BPA.xlsx': {'metadata': {'name': 'PLANO DE AÇÃO - BPA.xlsx', 'type': 'appli…

Button(button_style='success', description='Converter e Baixar', style=ButtonStyle())

✅ Conversão concluída com sucesso!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>